In [1]:
from python_graphql_client import GraphqlClient

In [4]:
endpoint = 'https://api2.mapofzones.com/v1/graphql'

# Instantiate the client with an endpoint.
client = GraphqlClient(endpoint=endpoint)

# Create the query string and variables required for the request.
query = """
    query ZoneName($zone: String!) {
        blockchain: flat_blockchains(where: {network_id: {_eq: $zone}}) {
            name
            __typename
        }
    }
"""

variables = {"zone":"stride-1"}

# Synchronous request
data = client.execute(query=query, variables=variables)
print(data) 

{'data': {'blockchain': [{'name': 'Stride', '__typename': 'flat_blockchains'}]}}


In [10]:
query = """
query ZonesMap($period: Int!, $isMainnet: Boolean!) {
  zonesStats: flat_blockchains(where: {is_mainnet: {_eq: $isMainnet}}) {
    ...ZoneBaseInfoV2
    isMainnet: is_mainnet
    switchedStats: blockchain_switched_stats(
      where: {timeframe: {_eq: $period}, is_mainnet: {_eq: $isMainnet}}
    ) {
      ibcVolume: ibc_cashflow
      ibcVolumeIn: ibc_cashflow_in
      ibcVolumeOut: ibc_cashflow_out
      ibcVolumeRating: ibc_cashflow_rating
      ibcTransfersRating: ibc_transfers_rating
      dauRating: active_addresses_cnt_rating
      totalTxsRating: txs_rating
      __typename
    }
    __typename
  }
  zonesGraphs: flat_blockchain_relations(
    where: {blockchain: {is_mainnet: {_eq: $isMainnet}}, blockchainByBlockchainSource: {is_mainnet: {_eq: $isMainnet}}, timeframe: {_eq: $period}}
  ) {
    source: blockchain_source
    target: blockchain_target
    ibcVolume: ibc_cashflow
    __typename
  }
}

fragment ZoneBaseInfoV2 on flat_blockchains {
  zone: network_id
  logoUrl: logo_url
  name: name
  __typename
}
"""

variables = {"period": 24, "isMainnet": True}
data = client.execute(query=query, variables=variables)

import json
with open('data/zonemap.json', 'w') as f:
    json.dump(data, f)